In [1]:
import pandas as pd
from jiwer import cer, wer
from tqdm import tqdm
import numpy as np

In [2]:
whisper_preds_path='/Users/markjos/projects/malachor5/data/pyarrow-datasets/tira-clean-split/tira-val-whisper.csv'
df=pd.read_csv(whisper_preds_path, keep_default_na=False)
df.head()

,path,english,german,spanish,russian,korean,french,japanese,portuguese,turkish,...,sinhala,albanian,turkmen,tatar,yiddish,latin,nynorsk,sanskrit,hawaiian,sundanese
0,HH01082021-m01s25ms035-m01s26ms975.wav,and I'll catch you later.,und ich habe das,y yo que chelo,И я качал,그리고...,et je vais qu'est-ce que l'on,そして、,E eu que chego,ve o da,...,අපි අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Ia ke qe lo,Ia ke chelo,И јо кешело,"יא, קצ'לו.",y yoha ke chelo,ლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლლ...,"y yoh, ke che lo",y yaw ke chelo,Iwa ke chelo
1,HH01082021-m01s42ms440-m01s43ms330.wav,Kitchelow,Kichelow,que chaval,Качало!,끝이 없네,Qu'est ce que je veux?,ケチャロン,Que chalo!,Kötü çalar,...,අපි කින් කින් කින් කින් කින් කින් කින් කින් ක...,Ke qe ljoh,qe chalo,Кетчало!,קצ' שלו,kê chalo?,ស្្្្្,kê chalo,ស្្្្្្្្្,Ketjalo
2,HH01082021-m01s45ms710-m01s46ms770.wav,Ketchup,Ketchup,que echalo,Кэчело,그쵸?,qu'est ce que je l'ai,きっちょの,quecham-o,Kötü,...,අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Ke qe l'o,qe chelo,кетчало,קצ'לו,ស្្្្្្្្្,ស្្្្,វ្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្្...,ស្្្្្្្្,ke chelo
3,HH01082021-m01s59ms245-m02s01ms005.wav,and what's up?,Und jetzt geht es los!,y yo que chelo,"И о, качалу!","이어, 그 체로","Et au-delà de ce que je l'ai fait,",いいよー!けっちょろ!,E ó que chalo,Yaa keçelo,...,අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Ijo ke qe qe lo,"Iyo, ke chelo.",И јо кетчало.,"יא, קצ'לו.",yoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoy...,ḍᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃᵃ...,y y y y y y y y y y y y y y y y y y y y y y y...,yoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoyoy...,Ia ke chelo
4,HH01082021-m02s32ms780-m02s34ms960.wav,"After that, this is the end.",dann die Kälte,apre y hechelo,"А после, это все.",그리고 이 부분을,"après, il y a le chelon",アプリ、イエチロ,apre e echelo,sonra bu çelik,...,අපි අපි කිර්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න්න...,Apre je qe lo,apre yechelo,апре ечело.,הפרי יצלו,apre ye chelo,"after that, it's done",apre ye chelo,apre ye chelo,apre ye chelo


In [5]:
tira_csv_path='/Users/markjos/projects/malachor5/data/pyarrow-datasets/tira-clean-split/metadata.csv'
tira_df=pd.read_csv(tira_csv_path)
tira_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/markjos/projects/malachor5/data/tira-clean-split/metadata.csv'

In [19]:
tira_df=tira_df[tira_df['split']=='validation']
df.shape, tira_df.shape

((2048, 100), (2048, 32))

In [20]:
df['tira']=tira_df['transcription'].to_list()
df['tira']

0         ìjɔ̀ kə̀cə̀lò
1               kə̀cə̀lò
2               kə̀cə̀lò
3         ìjɔ̀ kə̀cə̀lò
4        àprí jə̀cə̀lò
              ...        
2043       ðál ðô ìdú
2044       ðál ðô ìdú
2045    lál lɛ́ré ídú
2046    lál lɛ́ré ídú
2047    lál lɛ́ré ídú
Name: tira, Length: 2048, dtype: object

In [48]:
yoruba_cer=cer(df['tira'].tolist(), df['yoruba'].tolist())
yoruba_cer

5.523120106367262

In [33]:
(df['tira'].str.strip()=='').value_counts()

tira
False    2048
Name: count, dtype: int64

In [49]:
metrics={}
for column in tqdm(df.columns):
    if column=='tira' or column=='path':
        continue
    # avoid errors from empty strings
    df[column]=df[column].apply(lambda s: 'q' if s.strip()=='' else s)
    col_cer=cer(df['tira'].tolist(), df[column].tolist())
    col_wer=wer(df['tira'].tolist(), df[column].tolist())
    metrics[column]={'wer': col_wer, 'cer': col_cer}
metrics

100%|██████████| 101/101 [00:20<00:00,  4.83it/s]


{'english': {'wer': 2.2033188065705667, 'cer': 1.2451301099550471},
 'german': {'wer': 2.15135769359705, 'cer': 1.3352468184792015},
 'spanish': {'wer': 2.7018102581294, 'cer': 1.4371187978810966},
 'russian': {'wer': 1.8027153871941, 'cer': 1.3555705632821897},
 'korean': {'wer': 3.71656050955414, 'cer': 2.143194816706414},
 'french': {'wer': 2.8809922896413007, 'cer': 1.6664204461515733},
 'japanese': {'wer': 1.0509554140127388, 'cer': 1.6655551569128169},
 'portuguese': {'wer': 5.3697619845792826, 'cer': 2.276027267163329},
 'turkish': {'wer': 2.163090848139457, 'cer': 1.3447016862587848},
 'polish': {'wer': 2.154877639959772, 'cer': 1.2094633096258152},
 'catalan': {'wer': 5.544753603754609, 'cer': 2.7039233480362155},
 'dutch': {'wer': 2.162923231645994, 'cer': 1.2691893717155942},
 'arabic': {'wer': 2.9210526315789473, 'cer': 1.9232636177532025},
 'swedish': {'wer': 2.139792155548106, 'cer': 1.243779414557964},
 'italian': {'wer': 2.9664767013074087, 'cer': 1.5648650359833696},
 

In [55]:
min_lang=min(metrics, key=lambda k: metrics[k]['cer'])
min_lang, metrics[min_lang]


('croatian', {'wer': 1.4627891384512235, 'cer': 0.7631217947365089})

In [58]:
df['croatian']

0           Iga, kečelo.
1              Kaj čelo?
2              Kaj čelo?
3        I joj, ke čelo?
4         A pre je čelo.
              ...       
2043        da ldo i ru.
2044      zalvo i dobro.
2045         Galle Redu.
2046         Nalle Redu.
2047     Zala le re edu.
Name: croatian, Length: 2048, dtype: object

In [60]:
metrics_df=pd.DataFrame(metrics).transpose()
metrics_df.head()

,wer,cer
english,2.203319,1.245130
german,2.151358,1.335247
spanish,2.701810,1.437119
russian,1.802715,1.355571
korean,3.716561,2.143195


In [64]:
metrics_df.sort_values('cer')[:20]

,wer,cer
croatian,1.462789,0.763122
sundanese,1.349480,0.770276
slovenian,1.485753,0.784205
bosnian,1.497318,0.791719
serbian,1.557995,0.834244
finnish,1.366410,0.852479
lithuanian,1.440329,0.856932
maori,1.624874,0.866598
malagasy,1.434294,0.877213
estonian,1.540228,0.913281
